# Calibration using graphs

In [6]:
%matplotlib inline

import os
import sys

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from functools import partial
from dask import delayed
from distributed import progress
import dask.bag as bag

sys.path.append(os.path.join('..', '..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.calibration.solvers import solve_gaintable
from arl.calibration.operations import apply_gaintable
from arl.data.data_models import Image
from arl.data.polarisation import PolarisationFrame
from arl.data.parameters import get_parameter
from arl.visibility.base import create_blockvisibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.deconvolution import deconvolve_cube
from arl.image.operations import show_image
from arl.image.iterators import  image_raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration
from arl.imaging import predict_2d, advise_wide_field
    
from arl.graphs.dask_init import get_dask_Client
from arl.imaging.imaging_context import imaging_context, invert_function, predict_function
from arl.graphs.delayed import create_invert_graph, create_predict_graph, \
    create_selfcal_graph_list
from arl.graphs.generic_graphs import create_generic_image_graph
from arl.util.delayed_support import create_simulate_vis_graph, \
    create_predict_gleam_model_graph, create_corrupt_vis_graph
from arl.pipelines.delayed import create_continuum_imaging_pipeline_graph
from arl.graphs.vis import simple_vis

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

In [7]:
c=get_dask_Client()

Creating LocalCluster and Dask Client
<Client: scheduler='tcp://127.0.0.1:60567' processes=8 cores=8>
Diagnostic pages available on port http://127.0.0.1:8787


We create a graph to make the visibility 

In [8]:
nfreqwin=3
ntimes=5
frequency=numpy.linspace(0.8e8,1.2e8,nfreqwin)
if nfreqwin > 1:
    channel_bandwidth=numpy.array(nfreqwin*[frequency[1]-frequency[0]])
else:
    channel_bandwidth=numpy.array([1e7])
times = numpy.linspace(-numpy.pi/3.0, numpy.pi/3.0, ntimes)
phasecentre=SkyCoord(ra=+30.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox='J2000')

vis_graph_list=create_simulate_vis_graph('LOWBD2-CORE',
                                         frequency=frequency, 
                                         channel_bandwidth=channel_bandwidth,
                                         times=times,
                                         phasecentre=phasecentre)

DEBUG:arl.util.testing_support:create_configuration_from_file: Maximum radius 1000.0 m includes 166 antennas/stations


create_configuration_from_file: Maximum radius 1000.0 m includes 166 antennas/stations


DEBUG:arl.util.delayed_support:create_simulate_vis_graph: Simulating distribution in frequency


create_simulate_vis_graph: Simulating distribution in frequency


Find the optimum values for wide field imaging

In [9]:
wprojection_planes=1
advice=advise_wide_field(vis_graph_list[0].compute(), guard_band_image=4.0, delA=0.02,
                         wprojection_planes=wprojection_planes)
vis_slices = advice['vis_slices']

DEBUG:arl.visibility.coalesce:convert_visibility: Original 5 rows, 0.006 GB, converted 68475 rows, 0.007 GB


convert_visibility: Original 5 rows, 0.006 GB, converted 68475 rows, 0.007 GB


INFO:arl.imaging.params:advise_wide_field: Maximum wavelength 3.747 (meters)


advise_wide_field: Maximum wavelength 3.747 (meters)


INFO:arl.imaging.params:advise_wide_field: Minimum wavelength 3.747 (meters)


...

advice_wide_field: Number of planes in w stack 37


INFO:arl.imaging.params:advice_wide_field: Number of planes in w projection 1


advice_wide_field: Number of planes in w projection 1


INFO:arl.imaging.params:advice_wide_field: Recommend that 2d gridding (i.e. no wprojection) is used


advice_wide_field: Recommend that 2d gridding (i.e. no wprojection) is used


Now make a graph to fill with a model drawn from GLEAM. We then add phase errors of off 1 radian rms to each station. We will compute this graph in order not to confuse it's processing with the imaging. 

In [11]:
corrupted_vis_graph_list = create_predict_gleam_model_graph(vis_graph_list,
                                                            frequency=[frequency[len(frequency)//2]],
                                                            channel_bandwidth=[channel_bandwidth[len(frequency)//2]],
                                                            context='wstack',
                                                                vis_slices=vis_slices)
corrupted_vis_graph_list = create_corrupt_vis_graph(corrupted_vis_graph_list, 
                                                    phase_error=1.0)
corrupted_vis_graph_list=c.compute(corrupted_vis_graph_list)

Now make a graph to construct the LSM. The LSM is drawn from GLEAM as well but only includes sources brighter than 1 Jy

In [19]:
print(advice.keys())
from arl.util.bag_support import create_low_test_image_from_gleam
npixel = advice['npixels2']
cellsize = advice['cellsize']
gleam_model_graph = [
    delayed(create_low_test_image_from_gleam)(
        npixel=npixel,
        frequency=[frequency[f]],
        channel_bandwidth=[channel_bandwidth[f]],
        cellsize=cellsize,
        phasecentre=phasecentre,
        polarisation_frame=PolarisationFrame("stokesI"),
        flux_limit=0.1,
        applybeam=True) for f, freq in enumerate(frequency)
]

dict_keys(['kernel', 'vis_slices', 'wstep', 'freq_sampling_primary_beam', 'freq_sampling_image', 'time_sampling_primary_beam', 'time_sampling_image', 'w_sampling_primary_beam', 'w_sampling_image', 'npixels2', 'npixels', 'pwr23', 'cellsize', 'synthesized_beam', 'facet_fov', 'image_fov', 'primary_beam_fov', 'diameter', 'maximum_baseline', 'min_wavelength', 'max_wavelength', 'svis', 'wprojection_planes', 'facets', 'guard_band_image', 'oversampling_synthesised_beam', 'delA', 'vis'])


Now make a dirty image to see the effect of the phase errors introduced

In [24]:
dirty_graph = create_invert_graph(corrupted_vis_graph_list, gleam_model_graph, context='wstack',
                                         vis_slices=vis_slices, dopsf=False)
print(dirty_graph)
future=c.compute(dirty_graph)
print(future[0].result())
dirty=future.result()[0]
show_image(dirty, title='No selfcal')
plt.show()

[Delayed('sum_invert_results-2d466902-6247-4672-ba40-3a67595b1bae'), Delayed('sum_invert_results-84701943-efbf-40b3-b7fe-e84d6aab3f1d'), Delayed('sum_invert_results-7d2d7dbc-ebf3-4d6e-aa30-583141fe8e56')]


TypeError: create_low_test_image_from_gleam() got multiple values for argument 'npixel'

First make a selfcal graph in which the different Visibility's are selfcal'ed independently. We will look at the graph for just one Visibility.

In [ ]:
selfcal_vis_graph_list = create_selfcal_graph_list(corrupted_vis_graph_list, LSM_graph, context='wstack',
                                                   vis_slices=vis_slices,
                                                   global_solution=False)
simple_vis(selfcal_vis_graph_list[0])

The graph for making the dirty image now shows a global synchronisation point. We alleviate this by only sending averaged visibilities to the gather step before averaging over the solution interval, the model visibility is divided out. Only the gaintable is sent back for application.

In [ ]:
dirty_graph = create_invert_graph(selfcal_vis_graph_list, LSM_graph, context='wstack',
                                         vis_slices=vis_slices, dopsf=False)
simple_vis(dirty_graph)

In [ ]:
future=c.compute(dirty_graph)
dirty=future.result()[0]
show_image(dirty, title='With selfcal')
plt.show()

In [ ]:
c.shutdown()